<h1 style="color:gold; background-color:black; padding:20px; font-size:40px; text-align: center">TRADUCTOR QUECHUA - ESPAÑOL</h1>

<h1>Importar librerias a utilizar</h1>

In [1]:
import json
import numpy as np
import os
import pandas as pd

In [2]:
import pydot
import graphviz

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

<h1>Definir funciones auxiliares</h1>

<h3 style="color:crimson">Imprime barra de progreso</h3>

In [4]:
def imprimir_barra_progreso_v2(progreso, total):
    # https://www.youtube.com/watch?v=x1eaT88vJUA
    
    p = int(100 * (progreso + 1)/total)

    # Alt+291: █
    barra = '█'*p + '-'*(100-p) 

    print("\r|{}| {}%".format(barra, p), end='\r')

<h1>Importar datos</h1>

<p>Los archivos contienen palabras/oraciones paralelas
<br>
La primera columna está en español y la segunda en quechua</p>

In [5]:
# Crear un dataframe vació que almacenará las traducciones (translations)
df_trans = pd.DataFrame(columns=["español","quechua"])
df_trans

,español,quechua


<p>Se utilizó un tab ("\t") como separador de columnas para evitar confusiones con las comas propias de las oraciones</p>

<h3 style="color:crimson">Importa DataFrames</h3>

In [6]:
arr_opciones = [[40,[40,50,60]],
               [50,[50,63,75]],
               [60,[60,75,90]]]

In [7]:
print("Longitudes disponibles en español")
for i in range(3):
    print("{}) - {}".format(i, arr_opciones[i][0]) )
MAX_ES_index = int(input("Ingrese la longitud máxima en español: "))

print("\n" + "-"*100)
print("Longitudes disponibles en quechua")
for j in range(3):
    print("{}) - {}".format(j, arr_opciones[MAX_ES_index][1][j]))
MAX_QU_index = int(input("Ingrese la longitud máxima en quechua: "))

Longitudes disponibles en español
0) - 40
1) - 50
2) - 60
Ingrese la longitud máxima en español: 0

----------------------------------------------------------------------------------------------------
Longitudes disponibles en quechua
0) - 40
1) - 50
2) - 60
Ingrese la longitud máxima en quechua: 0


In [8]:
MAX_ES = arr_opciones[MAX_ES_index][0]
MAX_QU = arr_opciones[MAX_ES_index][1][MAX_QU_index]

folder_consolidado = "./Datos/consolidado_{}_{}/".format(MAX_ES, MAX_QU)
print("obteniendo datos de {}".format(folder_consolidado))

sub_paths = [folder_consolidado,
             #"./Datos/palabras/", "./Datos/grupos/",
             #"./Datos/libros/", "./Datos/oraciones/"
            ]

for sub_path in sub_paths:
    arr_category = os.listdir(sub_path)

    for item in arr_category:
        if item[-4:] == ".csv":
            file = "{}{}".format(sub_path,item)
            df_temp = pd.read_csv(file, encoding="utf-8", sep="\t")
            
            df_trans = pd.concat([df_trans, df_temp]
                                 , ignore_index = True)

obteniendo datos de ./Datos/consolidado_40_40/


In [9]:
df_trans.head(100)

,español,quechua
0,Ve.,Riy.
1,Vete.,Lluqsiy.
2,Vaya.,Waw.
3,Váyase.,Lluqsiy.
4,Hola.,Allinllachu.
...,...,...
95,Me escapé.,Ayqirqanim.
96,Huía.,ayqirqa
97,Me escapaba.,Ayqichkarqanim.
98,Yo lo sé.,Chaytaqa yachani.


In [10]:
df_trans.tail(100)

,español,quechua
90213,Como un padre decrépito disfruta,Imaynan huk decrepit tayta kusikun
90214,"al ver de su hijo las empresas jóvenes,","churinpa wayna empresankunata rikuspa,"
90215,"así yo, mutilado por la suene,","chaymi ñoqa, t’oqyaywan mutilado, ."
90216,"el poder o el ingenio, uno o todos,","atiy utaq ingenio, huk utaq llapan, ."
90217,"para colmarme a mí con su opulencia,","qhapaq kayninwan hunt'achiwananpaq,"
...,...,...
90308,"Pobre alma, centro de culpable limo","Wakcha alma, huchayuq limo centro"
90309,"a la que burla, indócil, quien la ciñe,","burlakuq, mana kamachikuq, chumpikuq, ."
90310,ornando tu morada pasajera?,temporal tiyasqaykita sumaqchaspa?
90311,"Vive, alma, a expensas de tu servidor;","Kawsay, alma, kamachiykiq qolqenwan;"


<h3 style="color:crimson">Eliminar frases muy largas</h3>
<p>Los tres arreglos de numpy deben entrar en los 8 GB de VRAM, y dejar espacio para el resto de variables</p>

In [11]:
print("Longitud inicial del arreglo {}".format(len(df_trans)))

Longitud inicial del arreglo 90313


In [12]:
for index, registro in df_trans.iterrows():
    txt_es = registro[0].strip()
    txt_qu = registro[1].strip()
    
    if (len(txt_es) > MAX_ES) or (len(txt_qu) > MAX_QU):
        #print("registro eliminado")
        df_trans = df_trans.drop(index)
        
    if index % 5000 == 0:
        print("Registro {} analizado".format(index))

Registro 0 analizado
Registro 5000 analizado
Registro 10000 analizado
Registro 15000 analizado
Registro 20000 analizado
Registro 25000 analizado
Registro 30000 analizado
Registro 35000 analizado
Registro 40000 analizado
Registro 45000 analizado
Registro 50000 analizado
Registro 55000 analizado
Registro 60000 analizado
Registro 65000 analizado
Registro 70000 analizado
Registro 75000 analizado
Registro 80000 analizado
Registro 85000 analizado
Registro 90000 analizado


In [13]:
print("Nueva longitud del arreglo {}".format(len(df_trans)))

Nueva longitud del arreglo 90313


<h1>Configuración</h1>

In [14]:
batch_size = 128 # tamño de los lotes para entrenamiento
epochs = 250 # Número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = len(df_trans)

<h1>Preparar los datos</h1>

In [15]:
# Vectoriza los datos
i=0
target_text= ''

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [16]:
for index, registro in df_trans.iterrows():    
    # Notar que los índices están al revés
    input_text = registro[1].strip()
    target_text = registro[0].strip()
    
    
    if (index<100):
        pad_dashes = " " + "-"*(20-len(input_text)) + " "
        print("{} \t I: |{}| {} T: |{}|"
              .format(index, input_text, pad_dashes, target_text))        

    
    # Usaremos "tab" como el  caracter de inicio (start sequence)
    # para los targets, y "\n" como el caracter de fin de secuencia "end sequence"
    target_text = "\t" + target_text + "\n"
    # sube las líneas a  las listas
    input_texts.append(input_text)
    target_texts.append(target_text)
  
    # completa los conjuntos de caracteres si es necesario
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    i=i+1

0 	 I: |Riy.|  ----------------  T: |Ve.|
1 	 I: |Lluqsiy.|  ------------  T: |Vete.|
2 	 I: |Waw.|  ----------------  T: |Vaya.|
3 	 I: |Lluqsiy.|  ------------  T: |Váyase.|
4 	 I: |Allinllachu.|  --------  T: |Hola.|
5 	 I: |¡Runa!|  --------------  T: |¡Corre!|
6 	 I: |Paway!|  --------------  T: |¡Corran!|
7 	 I: |¡Ascapa!|  ------------  T: |¡Huye!|
8 	 I: |Paway!|  --------------  T: |¡Corra!|
9 	 I: |Paway!|  --------------  T: |¡Corred!|
10 	 I: |Paway.|  --------------  T: |Corra.|
11 	 I: |Paway.|  --------------  T: |Corred.|
12 	 I: |Pi?|  -----------------  T: |¿Quién?|
13 	 I: |Waw!|  ----------------  T: |¡Órale!|
14 	 I: |Kumuy!|  --------------  T: |¡Inclínate!|
15 	 I: |Nina!|  ---------------  T: |¡Fuego!|
16 	 I: |Nina!|  ---------------  T: |¡Incendio!|
17 	 I: |disparar!|  -----------  T: |¡Disparad!|
18 	 I: |Yanapay!|  ------------  T: |¡Ayuda!|
19 	 I: |Yanapay! Yanapay!|  ---  T: |¡Socorro! ¡Auxilio!|
20 	 I: |Yanapay!|  ------------  T: |¡Auxilio!|
21 	 I: |

In [17]:
# Convierte los dos conjuntos de caracteres
# en dos listas ordenadas
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))  
# calcule el número de tokens (caracteres) en ambos lados
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# calcula la máxima longitud de las secuencias en cada lado
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

In [18]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("preparando datos...")

Number of samples: 90313
Number of unique input tokens: 103
Number of unique output tokens: 111
Max sequence length for inputs: 40
Max sequence length for outputs: 42
preparando datos...


In [19]:
# crea diccionarios de tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# crea los tensores 1-hot para el encoder y el decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Modificar los valores de la matriz del encoder
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print ("\n....")
print("datos preparados")


....
datos preparados


<h1>Construir el modelo</h1>

<h3 style="color:crimson">Encoder</h3>

In [21]:
# define una secuencia de entrada y la procesa
encoder_inputs = Input(shape = (None, num_encoder_tokens))

# capa recurrente del encoder
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Descartamos las salidas (encoder_outputs)
# solamente se conserva las memoria de  corto (state_h) y 
# largo plazo(state_c)
encoder_states = [state_h, state_c]

<h3 style="color:crimson">Decoder</h3>

In [22]:
# Configuramos el decoder, usando 'encoder_states' como estado inicial
decoder_inputs = Input(shape= (None, num_decoder_tokens))

# capa recurrente del decoder
# Configuramos nuestro decodificador para devolver secuencias de salida completas,
# y también para devolver estados internos. No usamos los
# estados retornados en el modelo de entrenamiento, pero los usaremos en inferencia.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

<h3 style="color:crimson">Modelo Completo</h3>

In [23]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [24]:
model.summary()
plot_model(model, to_file='../Imagenes/s2s.png', 
           show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 103)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 111)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        368640      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

<h1>Entrenar el modelo</h1>

In [25]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [26]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/250
565/565 [==============================] - 19s 28ms/step - loss: 1.5552 - accuracy: 0.5762 - val_loss: 1.7008 - val_accuracy: 0.5091
Epoch 2/250
565/565 [==============================] - 14s 25ms/step - loss: 1.1518 - accuracy: 0.6484 - val_loss: 1.5465 - val_accuracy: 0.5557
Epoch 3/250
565/565 [==============================] - 14s 25ms/step - loss: 1.0500 - accuracy: 0.6750 - val_loss: 1.4399 - val_accuracy: 0.5802
Epoch 4/250
565/565 [==============================] - 14s 25ms/step - loss: 0.9672 - accuracy: 0.7014 - val_loss: 1.3678 - val_accuracy: 0.6021
Epoch 5/250
565/565 [==============================] - 14s 25ms/step - loss: 0.9323 - accuracy: 0.7128 - val_loss: 1.3152 - val_accuracy: 0.6169
Epoch 6/250
565/565 [==============================] - 14s 25ms/step - loss: 0.8721 - accuracy: 0.7308 - val_loss: 1.2702 - val_accuracy: 0.6331
Epoch 7/250
565/565 [==============================] - 14s 26ms/step - loss: 0.8307 - accuracy: 0.7440 - val_loss: 1.2345 - val_ac

565/565 [==============================] - 14s 25ms/step - loss: 0.2516 - accuracy: 0.9219 - val_loss: 1.3432 - val_accuracy: 0.7285
Epoch 114/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2515 - accuracy: 0.9217 - val_loss: 1.3556 - val_accuracy: 0.7274
Epoch 115/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2503 - accuracy: 0.9222 - val_loss: 1.3513 - val_accuracy: 0.7286
Epoch 116/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2501 - accuracy: 0.9223 - val_loss: 1.3613 - val_accuracy: 0.7263
Epoch 117/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2496 - accuracy: 0.9224 - val_loss: 1.3615 - val_accuracy: 0.7278
Epoch 118/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2480 - accuracy: 0.9229 - val_loss: 1.3651 - val_accuracy: 0.7272
Epoch 119/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2470 - accuracy: 0.9232 - val_loss: 1.3745 - val_ac

Epoch 169/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2241 - accuracy: 0.9295 - val_loss: 1.5065 - val_accuracy: 0.7233
Epoch 170/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2233 - accuracy: 0.9297 - val_loss: 1.5165 - val_accuracy: 0.7221
Epoch 171/250
565/565 [==============================] - 15s 26ms/step - loss: 0.2232 - accuracy: 0.9298 - val_loss: 1.5204 - val_accuracy: 0.7220
Epoch 172/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2224 - accuracy: 0.9299 - val_loss: 1.5207 - val_accuracy: 0.7224
Epoch 173/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2221 - accuracy: 0.9302 - val_loss: 1.5207 - val_accuracy: 0.7230
Epoch 174/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2217 - accuracy: 0.9301 - val_loss: 1.5273 - val_accuracy: 0.7217
Epoch 175/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2209 - accuracy: 0.9303 - val_loss: 1

Epoch 225/250
565/565 [==============================] - 14s 24ms/step - loss: 0.2088 - accuracy: 0.9337 - val_loss: 1.6235 - val_accuracy: 0.7183
Epoch 226/250
565/565 [==============================] - 14s 25ms/step - loss: 0.2092 - accuracy: 0.9336 - val_loss: 1.6258 - val_accuracy: 0.7175
Epoch 227/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2082 - accuracy: 0.9340 - val_loss: 1.6212 - val_accuracy: 0.7194
Epoch 228/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2083 - accuracy: 0.9340 - val_loss: 1.6128 - val_accuracy: 0.7205
Epoch 229/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2086 - accuracy: 0.9338 - val_loss: 1.6131 - val_accuracy: 0.7197
Epoch 230/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2071 - accuracy: 0.9342 - val_loss: 1.6227 - val_accuracy: 0.7194
Epoch 231/250
565/565 [==============================] - 14s 26ms/step - loss: 0.2075 - accuracy: 0.9340 - val_loss: 1

<h1>Guardar el modelo</h1>

<h3 style="color:crimson">Diccionarios</h3>

In [27]:
with open("./Modelos/q2s/input_token_index.txt", "w") as f:
    json.dump(input_token_index, f)

In [28]:
with open("./Modelos/q2s/target_token_index.txt", "w") as f:
    json.dump(target_token_index, f)

In [29]:
with open("./Modelos/q2s/history.txt", "w") as f:
    json.dump(history.history, f)

<h3 style="color:crimson">Valores puntuales</h3>

In [30]:
print("num_decoder_tokens: {}".format(num_decoder_tokens))
print("max_decoder_seq_length: {}".format(max_decoder_seq_length))

print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

num_decoder_tokens: 111
max_decoder_seq_length: 42
Number of unique input tokens: 103
Number of unique output tokens: 111
Max sequence length for inputs: 40
Max sequence length for outputs: 42


In [31]:
with open("./Modelos/q2s/otros.txt", "w") as f:
    f.write("num_encoder_tokens = {}\n".format(num_encoder_tokens))
    f.write("num_decoder_tokens = {}\n".format(num_decoder_tokens))
    
    f.write("max_encoder_seq_length = {}\n".format(max_encoder_seq_length))
    f.write("max_decoder_seq_length = {}".format(max_decoder_seq_length))

<h3 style="color:crimson">Arreglo numpy</h3>
<p>Sólo las primeras 100 posiciones</p>

In [32]:
file_name = "./Modelos/q2s/encoder_input_data_sample.npy"
with open(file_name, "wb") as f:
    np.save(f, encoder_input_data[:100])

<h3 style="color:crimson">Modelo</h3>

In [33]:
# s2q = Spanish to Quechua
model.save("./Modelos/q2s/spanish_to_quechua")

INFO:tensorflow:Assets written to: ./Modelos/q2s/spanish_to_quechua\assets


INFO:tensorflow:Assets written to: ./Modelos/q2s/spanish_to_quechua\assets


In [34]:
model.save("./Modelos/q2s/spanish_to_quechua_file.h5")